In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from nltk import sent_tokenize, word_tokenize
from IPython.display import display
import re
import json
import pickle
import operator
import toolz

In [2]:
import sys

In [3]:
sys.path.append('../shared/')

from Argument import Argument

In [4]:
with open('../../not-gitted/argsme-crawled/1629700068.9873986-4566-arguments-cleaned.pickle', 'rb') as f:
        e = pickle.load(f)

In [6]:
e[0].sentence_embeddings.shape

(3, 768)

In [10]:
len(e)

4566

In [11]:
with open('../../not-gitted/argsme-crawled/1632239915.4824035-4390-arguments-cleaned-test.pickle', 'rb') as f:
        d = pickle.load(f)

In [12]:
d = list(toolz.unique(d, key=lambda a: a.arg_id+a.query))

In [13]:
len(d)

4390

In [15]:
e_ids = {a.arg_id for a in e}
d_ids = {a.arg_id for a in d}

In [16]:
e_ids == d_ids

False

In [17]:
print(len(e_ids), len(d_ids))

4439 3873


In [18]:
len(e_ids.intersection(d_ids))

555

In [22]:
only_test_ids = d_ids - e_ids

In [23]:
len(only_test_ids)

3318

In [24]:
d = set(filter(lambda a: a.arg_id in only_test_ids, d))

In [25]:
len(d)

3771

# Filter sentences

In [26]:
def clean(string):
    #regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))[.]*"
    regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    #url = list(re.findall(regex,string))
    #if len(url) > 0:
        #print([x for x in url])
    string = re.sub(regex, '', string)
    string = string.strip()
    string = re.sub(r'"', '', string)
    string = re.sub(r'[“”]', '', string)
    string = re.sub(r'\\', '', string)
    string = re.sub(r'[-]{2,}', '', string)
    string = re.sub(r'\[[0-9]\]','', string)
    string = re.sub(r'^[^\w(]+', '', string) # sentences that start with special characters (non-alpha chars, except '(')
    string = re.sub(r'\s{2,}', ' ', string)
    
    return string.strip()
    

def remove(string):
    regex = r"^[\[\(]?([0-9]+|[Ii]+)[\]\)]?\.?$"
    pattern = re.compile(regex)
    pattern2 = re.compile(r'^[\s]+$')
    return pattern.match(string) \
        or pattern2.match(string) \
        or len(string) < 3 \
        or len([c for c in string if c.isalpha()]) < 2 \
        or all(not c.isalpha() for c in string) \
        or string.lower().startswith('thank') \
        or string.lower().startswith('good luck') \
        or string.lower().startswith('sources') \
        or string.lower().startswith('references')

In [27]:
def fix_sentence_splitting(sentences):
    target_sentences = list()
    for i_s, s in enumerate(sentences):
        if s[0].isalpha() and s[0].islower() and i_s > 1:
            current_collection = ''
            if len(target_sentences) > 1:
                current_collection += target_sentences.pop()
                current_collection += ' ' + s
                target_sentences.append(current_collection)
            else:
                # nothing to append to
                target_sentences.append(s)
        else:
            target_sentences.append(s)

    return target_sentences

In [28]:
for a in tqdm(d):
    sents = list()
    for s in a.sentences:
        s = clean(s)
        r = remove(s)
        if not r:
            #print(s, r)
            sents.append(s)
    if len(sents) == 0 or all(len(t) == 0 for t in sents):
        print(f"No sentence argument: {a.arg_id}")
    else:
        sents = fix_sentence_splitting(sents)
    #print(sents)
    a.sentences = sents

100%|████████████████████████████████████████████████████████████████████████████| 3771/3771 [00:03<00:00, 1213.92it/s]


In [29]:
[a for a in d if a.arg_id == 'S47ca84a7-A120f3ad4'][0].sentences[-1]

IndexError: list index out of range

In [30]:
[a for a in d if a.arg_id == 'S939644dc-A309e66cd'][0].sentences#[13]#.lower().startswith('thank')

IndexError: list index out of range

In [31]:
len(d)

3771

In [32]:
filtered_arguments = list()
filtered_arg_ids = list()
for argument in tqdm(d):
    if len(argument.sentences) > 2:
        filtered_arguments.append(argument)
        filtered_arg_ids.append(argument.arg_id)

100%|██████████████████████████████████████████████████████████████████████████| 3771/3771 [00:00<00:00, 418952.68it/s]


In [33]:
len(filtered_arguments)

3761

# Contexts

In [34]:
discussion_title_arg_counts = dict()
for argument in tqdm(filtered_arguments):
    title = argument.query
    if title in discussion_title_arg_counts:
        discussion_title_arg_counts[title] +=1
    else:
        discussion_title_arg_counts[title] = 1

100%|██████████████████████████████████████████████████████████████████████████| 3761/3761 [00:00<00:00, 470131.05it/s]


In [35]:
contexts = pd.DataFrame.from_dict([{'title': k, 'count': v} for k, v in discussion_title_arg_counts.items()])

In [36]:
filtered_contexts = contexts[contexts['count'] >= 2]
filtered_contexts

,title,count
0,the_very_rich_employ_expert_accountants_and_la...,5
1,k-pop,8
2,violence,8
3,fracking,10
4,decision_theory_is_important,9
...,...,...
474,trade_network,4
475,star_wars,3
476,arsenal,7
477,home_schooling,4


In [37]:
filtered_contexts.describe()

,count
count,475.000000
mean,7.907368
std,2.035510
min,2.000000
25%,7.000000
50%,8.000000
75%,9.500000
max,10.000000


In [38]:
context_ids = set(filtered_contexts['title'])
len(context_ids)

475

In [39]:
ready_to_summarize = list()
for argument in tqdm(filtered_arguments):
    if argument.query in context_ids:
        ready_to_summarize.append(argument)

100%|██████████████████████████████████████████████████████████████████████████| 3761/3761 [00:00<00:00, 250851.19it/s]


In [40]:
len(ready_to_summarize)

3756

In [41]:
for a in ready_to_summarize:
    a.sentence_embeddings = list()

In [42]:
with open('../../not-gitted/argsme-crawled/1632239915.4824035-3756-arguments-cleaned-test.pickle', 'wb') as f:
        pickle.dump(ready_to_summarize, f)